# S3 Vector Memory for Persistent Context

Build semantic memory with [Amazon S3 Vectors](https://docs.aws.amazon.com/AmazonS3/latest/userguide/s3-vectors.html). This notebook demonstrates how to create agents that remember information across all sessions with production-ready infrastructure.

## What You'll Learn

- Implement cross-session memory with Amazon S3 Vectors
- Configure semantic search for agent knowledge
- Build production-ready persistent memory systems
- Compare session managers with vector memory approaches

## Prerequisites

- Completed [Notebook 04: State and Sessions](04-state-and-sessions.ipynb)
- AWS account with S3 Vectors access
- Understanding of vector databases and semantic search

📚 **Learn More**: [Amazon S3 Vectors Getting Started Guide](https://docs.aws.amazon.com/AmazonS3/latest/userguide/s3-vectors-getting-started.html)

## Setup

The `s3_vector_memory` tool automatically creates the necessary infrastructure:
1. ✅ S3 Vector bucket (created automatically if it doesn't exist)
2. ✅ Vector index (created automatically if it doesn't exist)
3. ⚠️ Ensure you have appropriate AWS permissions:
   - `s3vectors:CreateVectorBucket`
   - `s3vectors:DescribeVectorBucket`
   - `s3vectors:CreateIndex`
   - `s3vectors:DescribeIndex`
   - `s3vectors:PutVectors`
   - `s3vectors:QueryVectors`
   - `bedrock:InvokeModel` (for embeddings)

See: [S3 Vectors Getting Started Guide](https://docs.aws.amazon.com/AmazonS3/latest/userguide/s3-vectors-getting-started.html)

In [ ]:
!pip install strands-agents strands-agents-tools boto3 -q

In [ ]:
import boto3
import os
from strands import Agent
from strands.models import BedrockModel
from strands_tools import image_reader, file_read, use_llm
from video_reader_local import video_reader_local
from s3_memory import s3_vector_memory

print("✅ All imports successful!")

## Configuration

Configure your S3 Vectors environment.

**Note**: The vector bucket and index will be created automatically on first use if they don't exist!

In [ ]:
# S3 Vectors Configuration
# IMPORTANT: Set AWS_REGION first - all resources will be created in this region
os.environ['AWS_REGION'] = 'us-east-1'                                    # Your AWS region (bucket, index, and Bedrock will use this)

# IMPORTANT: Replace 'your-unique-bucket-name' with a unique bucket name
# S3 bucket names must be globally unique across all AWS accounts
# Example: 'my-company-vector-store-2024' or 'username-strands-vectors'
os.environ['VECTOR_BUCKET_NAME'] = 'your-unique-bucket-name'              # ⚠️ CHANGE THIS!
os.environ['VECTOR_INDEX_NAME'] = 'strands-multimodal'                    # Index name (can keep as is)
os.environ['EMBEDDING_MODEL'] = 'amazon.nova-2-multimodal-embeddings-v1:0'  # Amazon Nova Multimodal Embeddings

print(f"🌍 Configuration set for region: {os.environ['AWS_REGION']}")
print(f"📦 Vector bucket: {os.environ['VECTOR_BUCKET_NAME']}")
print(f"📊 Vector index: {os.environ['VECTOR_INDEX_NAME']}")
print(f"🤖 Embedding model: {os.environ['EMBEDDING_MODEL']}")

if os.environ['VECTOR_BUCKET_NAME'] == 'your-unique-bucket-name':
    print("\n⚠️  WARNING: Please change VECTOR_BUCKET_NAME to a unique name!")


In [ ]:
# User identification for memory isolation
USER_ID = "demo_user_reinvent"

print(f"👤 User ID: {USER_ID}")


## S3 Vector Memory Tool

The `s3_vector_memory` tool provides three operations:

1. **store**: Save information to vector memory
2. **retrieve**: Search for relevant memories
3. **list**: List all memories for a user

### Automatic Infrastructure Setup

On first use, the tool automatically:
- Creates the S3 Vector bucket if it doesn't exist
- Creates the vector index with optimal configuration (1024 dimensions, COSINE distance)
- Waits for the index to be ready

This means you can start using vector memory immediately without manual setup!

In [ ]:
# Test storing a memory
result = s3_vector_memory(
    action="store",
    content="User prefers Python for backend development and React for frontend.",
    user_id=USER_ID
)
print("Store result:", result)

In [ ]:
# Test retrieving memories
result = s3_vector_memory(
    action="retrieve",
    query="programming preferences",
    user_id=USER_ID
)
print("Retrieved memories:", result)

In [ ]:
result

## Creating an Agent with Vector Memory

Now let's create a multi-modal agent with persistent vector memory:

In [ ]:
# Setup Bedrock model
aws_region = os.environ.get('AWS_REGION', 'us-east-1')
session = boto3.Session(region_name=aws_region)
print(f"🌍 Using AWS region: {aws_region}")
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    boto_session=session
)

# System prompt for memory-enhanced agent
MEMORY_SYSTEM_PROMPT = """You are an AI assistant with persistent memory across sessions.

Your capabilities:
- **Multi-Modal Analysis**: Process images, documents, videos, and text
- **Persistent Memory**: Remember user preferences, insights, and context
- **Cross-Session Understanding**: Access memories from previous conversations
- **Semantic Search**: Find relevant information from past interactions

Memory Usage Guidelines:
1. Before responding, retrieve relevant memories using s3_vector_memory
2. Store important insights, preferences, and findings
3. Reference previous conversations when relevant
4. Build understanding over time

When processing content:
1. Retrieve relevant memories for context
2. Analyze the new content
3. Store key insights
4. Provide comprehensive responses using both new analysis and memory
"""

# Create agent with vector memory
memory_agent = Agent(
    model=bedrock_model,
    tools=[
        s3_vector_memory,  # Vector memory tool
        image_reader,      # Image processing
        file_read,         # Document processing
        video_reader_local,      # Video processing
        use_llm           # Advanced reasoning
    ],
    system_prompt=MEMORY_SYSTEM_PROMPT
)

print("✅ Memory-enhanced agent created!")
print("🧠 Memory backend: Amazon S3 Vectors")

## First Interaction: Establishing Preferences

In [ ]:
# First conversation - establish user preferences
response = memory_agent(
    f"""Hello! I'm a developer advocate at AWS working on AI and serverless technologies.
    I'm particularly interested in:
    - Multi-modal AI processing
    - Serverless architectures
    - AWS Bedrock and generative AI
    - Building production-ready AI systems
    
    Please save this information about my interests for future conversations.
    
    USER_ID: {USER_ID}"""
)

print(response)

## Image Analysis with Memory Context

In [ ]:
# Analyze an architectural diagram with memory context
response = memory_agent(
    f"""Analyze the architectural diagram in data-sample/diagram.jpg.
    
    Before analyzing:
    1. Check my memory for my interests and preferences
    2. Use that context to provide a personalized analysis
    
    After analysis:
    1. Store key architectural insights
    2. Note technologies that align with my interests
    
    My user ID: {USER_ID}
    
    Provide:
    - Architecture overview
    - Technology stack
    - Best practices observed
    - Recommendations based on my interests"""
)

print(response)

## Document Processing with Memory

In [ ]:
# Process a document with memory integration
response = memory_agent(
    f"""Process the document data-sample/Welcome-Strands-Agents-SDK.pdf.
    
    Memory-enhanced processing:
    1. Retrieve my interests from memory
    2. Process the document with that context
    3. Store key insights that relate to my interests
    4. Connect to our previous architectural discussion
    
    My user ID: {USER_ID}
    
    Focus on:
    - How this relates to what we discussed before
    - Features relevant to my interests
    - Practical applications
    - AWS service integrations"""
)

print(response)

## Cross-Session Understanding

The power of vector memory: Start a new session and the agent still remembers!

In [ ]:
# Create a NEW agent (simulating a new session)
new_session_agent = Agent(
    model=bedrock_model,
    tools=[s3_vector_memory, image_reader, file_read, video_reader_local, use_llm],
    system_prompt=MEMORY_SYSTEM_PROMPT
)

# Ask about previous conversations
response = new_session_agent(
    f"""What do you know about my interests and what have we discussed before?
    
    USER_ID: {USER_ID}"""
)

print("🆕 New Session Response:")
print(response)

## Memory Operations

Let's explore the memory operations in detail:

In [ ]:
# List all memories for the user
result = s3_vector_memory(
    action="list",
    user_id=USER_ID
)

print(f"📊 Total memories stored: {result['total_found']}")
print("\nMemories:")
for i, mem in enumerate(result.get('memories', []), 1):
    content = mem.get('memory', '')
    # Truncate long content
    display_content = content[:100] + '...' if len(content) > 100 else content
    print(f"\n{i}. {display_content}")
    print(f"   Timestamp: {mem.get('created_at', 'N/A')}")

In [ ]:
# Semantic search across memories
result = s3_vector_memory(
    action="retrieve",
    query="serverless and AI technologies",
    user_id=USER_ID,
    top_k=3
)

print("🔍 Semantic Search Results:")
for i, mem in enumerate(result.get('memories', []), 1):
    print(f"\n{i}. {mem.get('memory', '')}")
    print(f"   Relevance Score: {mem.get('similarity', 'N/A')}")

## How It Works

### Architecture Overview

1. **Infrastructure Setup (Automatic)**
   - Checks if S3 Vector bucket exists, creates if needed
   - Checks if vector index exists, creates if needed
   - Configures index with 1024 dimensions
   - Uses COSINE distance metric for similarity

2. **Embedding Generation**
   - Text is converted to vectors using Amazon Bedrock embeddings
   - Model: `amazon.nova-2-multimodal-embeddings-v1:0` (state-of-the-art)
   - Produces 1024-dimensional vectors
   - Supports multimodal embeddings (text, images, video, audio)

3. **Storage**
   - Vectors stored in Amazon S3 with S3 Vectors
   - Metadata includes user_id, timestamp, content
   - Unique keys per memory for tracking

4. **Retrieval**
   - Semantic search using vector similarity
   - Returns most relevant memories
   - Filtered by user_id for isolation

5. **Isolation**
   - User-specific memories via user_id
   - Secure and scalable
   - Multi-tenant ready

## Use Cases

### Customer Support
- Remember customer issues across sessions
- Provide consistent support
- Track resolution history

### Personal Assistants
- Learn user preferences over time
- Provide personalized recommendations
- Maintain context across days/weeks

### Content Analysis
- Remember insights from analyzed content
- Connect information across documents
- Build knowledge over time

### Development Tools
- Remember project context
- Track decisions and rationale
- Provide consistent guidance

## Cleanup (Optional)

To avoid ongoing charges, you can delete the S3 Vector index and bucket created in this notebook.

In [ ]:
import boto3

# Initialize S3 Vectors client
s3vectors_client = boto3.client('s3vectors', region_name=AWS_REGION)

# Delete the vector index
try:
    print(f"Deleting index: {VECTOR_INDEX_NAME}...")
    s3vectors_client.delete_index(
        VectorBucketName=VECTOR_BUCKET_NAME,
        IndexName=VECTOR_INDEX_NAME
    )
    print(f"✅ Index '{VECTOR_INDEX_NAME}' deleted successfully")
except Exception as e:
    print(f"❌ Error deleting index: {e}")

In [ ]:
# Delete the vector bucket
try:
    print(f"Deleting bucket: {VECTOR_BUCKET_NAME}...")
    s3vectors_client.delete_vector_bucket(
        VectorBucketName=VECTOR_BUCKET_NAME
    )
    print(f"✅ Bucket '{VECTOR_BUCKET_NAME}' deleted successfully")
except Exception as e:
    print(f"❌ Error deleting bucket: {e}")

print("\n⚠️  Note: Deletion may take a few moments to complete.")

## Summary

In this notebook, you learned:

✅ Limitations of built-in session managers

✅ How Amazon S3 Vectors enables cross-session understanding

✅ How to configure and use s3_vector_memory tool

✅ How to create memory-enhanced agents

✅ Semantic search across memories

✅ Production-ready memory architecture

✅ Real-world use cases

### Key Takeaways

1. **Vector memory enables true AI personalization**
2. **S3 Vectors provides production-ready infrastructure**
3. **Semantic search is more powerful than keyword matching**
4. **Cross-session understanding creates better user experiences**

### Next Steps

- Implement vector memory in your applications
- Experiment with different embedding models
- Build personalized AI experiences
- Scale to production with confidence!